<a href="https://colab.research.google.com/github/Dinesh123reddy/CrewAI_UseCases/blob/main/Travel_Advisor_Using_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Packages

In [1]:
!pip install crewai
!pip install crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 867.0 kB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 764.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 905.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 32.4 MB/s

📝 Test Case: Planning a Last-Minute Budget Trip to London

**Scenario:**

- You have only 3 days for the trip.

- Your budget is $1500 (including flights, stay, food, transport).

- You prefer historical sites over shopping.

- You need to find a hotel near public transport.

- You want a real-time weather forecast for London before finalizing the plan.

In [3]:
import os
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool

from google.colab import userdata
gemini_key=userdata.get('gemini_key')
serper_key=userdata.get('serper_key')
os.environ["GOOGLE_API_KEY"] = gemini_key
os.environ["SERPER_API_KEY"] = serper_key

# Initialize a search tool (to fetch real-time travel info)
search_tool = SerperDevTool()

# Define the AI Model
llm = LLM(model="gemini/gemini-1.5-flash",
          verbose=True,
          temperature=0.5,
          api_key=os.environ["GOOGLE_API_KEY"])

# 🧭 Travel Researcher Agent (Finds historical sites + weather)
researcher = Agent(
    role="Travel Researcher",
    goal="Find historical sites, public transport hotels, and real-time weather for {destination}.",
    verbose=True,
    memory=True,
    backstory="You are an expert travel researcher, providing up-to-date information about history-focused trips.",
    llm=llm,
    tools=[search_tool],  # Uses live search tool
    allow_delegation=True
)

# 💰 Budget Planner Agent (Ensures trip stays under $1500)
budget_planner = Agent(
    role="Budget Planner",
    goal="Find budget flights, hotels, and activities within {budget} for {destination}.",
    verbose=True,
    memory=True,
    backstory="You are a skilled budget analyst ensuring trips fit within financial constraints.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=True
)

# 🗺️ Itinerary Planner Agent (Creates a balanced 3-day plan)
itinerary_planner = Agent(
    role="Itinerary Planner",
    goal="Create a 3-day itinerary for {destination}, ensuring all historical sites are covered under {budget}.",
    verbose=True,
    memory=True,
    backstory="You are an expert in trip planning, ensuring travelers get the best experience within their budget.",
    llm=llm,
    tools=[search_tool],
    allow_delegation=False
)

# 📝 Travel Research Task
research_task = Task(
    description="Find the best historical sites, weather forecast, and public transport hotels for {destination}.",
    expected_output="A list of top historical sites, a real-time weather update, and 3 hotel options near public transport.",
    tools=[search_tool],
    agent=researcher
)

# 💲 Budget Estimation Task
budget_task = Task(
    description="Find budget flights, hotel options, and daily food/transport costs for {destination}. Ensure total cost stays under {budget}.",
    expected_output="A full cost breakdown (flights, hotel, food, attractions) ensuring a $1500 budget is maintained.",
    tools=[search_tool],
    agent=budget_planner
)

# 📅 Itinerary Planning Task
itinerary_task = Task(
    description="Plan a 3-day itinerary for {destination}, focusing on historical sites, budget constraints, and real-time weather conditions.",
    expected_output="A detailed 3-day plan, considering weather and budget constraints, with transport recommendations.",
    tools=[search_tool],
    agent=itinerary_planner
)

# 🚀 Crew Setup: All agents working together!
crew = Crew(
    agents=[researcher, budget_planner, itinerary_planner],
    tasks=[research_task, budget_task, itinerary_task],
    process=Process.sequential  # Runs tasks in sequence
)

# 🔥 Run the CrewAI Trip Advisor system for London with a $1500 budget
result = crew.kickoff(inputs={'destination': 'Kerala', 'budget': '50000 INR'})
print(result)

# Agent: Travel Researcher
## Task: Find the best historical sites, weather forecast, and public transport hotels for Kerala.


# Agent: Travel Researcher
## Thought: tool_code
Thought: I need to gather information about historical sites, weather, and hotels near public transport in Kerala.  I'll start by searching for historical sites.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"historical sites in Kerala\"}"
## Tool Output: 
{'searchParameters': {'q': 'historical sites in Kerala', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': '28 Historical Places In Kerala | Heritage Places 2024 - Holidify', 'link': 'https://www.holidify.com/collections/historical-places-in-kerala', 'snippet': 'Here is the list of 28 Historical Places in Kerala · 1. Mattancherry Palace, Kochi · 2. Padmanabhaswamy Temple, Trivandrum · 3. Thalassery Fort, Thalaserry ...', 'position': 1}, {'title': 'Kerala Historic Sites & Districts to Visit (2025) - Tr

ERROR:root:LiteLLM call failed: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.QuotaFailure",
        "violations": [
          {
            "quotaMetric": "generativelanguage.googleapis.com/generate_content_free_tier_requests",
            "quotaId": "GenerateRequestsPerMinutePerProjectPerModel-FreeTier",
            "quotaDimensions": {
              "location": "global",
              "model": "gemini-1.5-flash"
            },
            "quotaValue": "15"
          }
        ]
      },
      {
        "@type": "type.googleapis.com/google.rpc.Help",
        "links": [
          {
            "description": "Learn more about Gemini AP






 An unknown error occurred. Please check the details below.



# Agent: Budget Planner
## Thought: tool_code
Thought: I need to find budget flights and accommodation first.  I'll delegate the task of finding hotels near public transport to the Itinerary Planner.  Then I'll use Serper to search for budget flights.
## Using tool: Delegate work to coworker
## Tool Input: 
"{\"task\": \"Find three budget-friendly hotel options in Kerala near public transport.  Prioritize hotels with good online reviews and reasonable prices. Provide hotel names, locations, approximate prices per night, and links to online reviews (e.g., Google Reviews, TripAdvisor).\", \"context\": \"I need to plan a trip to Kerala with a budget of 50000 INR (approximately $620 USD).  The total cost including flights, hotels, food, and activities needs to stay within this budget.  The hotels should be easily accessible using public transport.\", \"coworker\": \"Itinerary Planner\"}"



ERROR:root:LiteLLM call failed: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.QuotaFailure",
        "violations": [
          {
            "quotaMetric": "generativelanguage.googleapis.com/generate_content_free_tier_requests",
            "quotaId": "GenerateRequestsPerMinutePerProjectPerModel-FreeTier",
            "quotaDimensions": {
              "model": "gemini-1.5-flash",
              "location": "global"
            },
            "quotaValue": "15"
          }
        ]
      },
      {
        "@type": "type.googleapis.com/google.rpc.Help",
        "links": [
          {
            "description": "Learn more about Gemini AP






 An unknown error occurred. Please check the details below.



RateLimitError: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.",
    "status": "RESOURCE_EXHAUSTED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.QuotaFailure",
        "violations": [
          {
            "quotaMetric": "generativelanguage.googleapis.com/generate_content_free_tier_requests",
            "quotaId": "GenerateRequestsPerMinutePerProjectPerModel-FreeTier",
            "quotaDimensions": {
              "model": "gemini-1.5-flash",
              "location": "global"
            },
            "quotaValue": "15"
          }
        ]
      },
      {
        "@type": "type.googleapis.com/google.rpc.Help",
        "links": [
          {
            "description": "Learn more about Gemini API quotas",
     

# Streamlit Application

In [4]:
!pip install streamlit
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.6 MB/s eta 0:00:00


In [5]:
import os
from google.colab import userdata

gemini_key = userdata.get('gemini_key')
serper_key = userdata.get('serper_key')

os.environ['GEMINI_API_KEY'] = gemini_key
os.environ['SERPER_API_KEY'] = serper_key

In [8]:
%%writefile app.py
import os
import streamlit as st
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool

# --- Initialize search tool ---
search_tool = SerperDevTool()

# --- Define AI Model ---
llm = LLM(model="gemini/gemini-1.5-flash",
          verbose=True,
          temperature=0.5,
          api_key=os.environ.get("GOOGLE_API_KEY", ""))

# --- Streamlit UI ---
st.title("🌍 AI-Powered Travel Planner")
st.markdown("**Plan your perfect trip with AI-powered insights!**")

destination = st.text_input("📍 Enter Destination:")
budget = st.text_input("💰 Enter Budget (INR):")

# --- Function to create AI Agents ---
def create_agents(destination, budget):
    researcher = Agent(
        role="Travel Researcher",
        goal=f"Find historical sites, public transport hotels, and real-time weather for {destination}.",
        backstory="You are an expert travel researcher, providing up-to-date information about history-focused trips.",
        verbose=True,
        memory=True,
        llm=llm,
        tools=[search_tool]
    )

    budget_planner = Agent(
        role="Budget Planner",
        goal=f"Find budget flights, hotels, and activities within {budget} for {destination}.",
        backstory="You are a skilled budget analyst ensuring trips fit within financial constraints.",
        verbose=True,
        memory=True,
        llm=llm,
        tools=[search_tool]
    )

    itinerary_planner = Agent(
        role="Itinerary Planner",
        goal=f"Create a 3-day itinerary for {destination}, ensuring all historical sites are covered under {budget}.",
        backstory="You are an expert in trip planning, ensuring travelers get the best experience within their budget.",
        verbose=True,
        memory=True,
        llm=llm,
        tools=[search_tool]
    )

    return researcher, budget_planner, itinerary_planner

# --- Generate Travel Plan Button ---
if st.button("🎯 Generate Travel Plan"):
    if not destination or not budget:
        st.error("⚠️ Please enter both a destination and budget.")
    else:
        st.info("⏳ Generating your AI-powered travel plan...")

        # Create AI Agents
        researcher, budget_planner, itinerary_planner = create_agents(destination, budget)

        # Define AI Tasks
        research_task = Task(
            description=f"Find the best historical sites, weather forecast, and public transport hotels for {destination}.",
            expected_output="✅ A list of top historical sites, a real-time weather update, and 3 hotel options near public transport.",
            tools=[search_tool],
            agent=researcher
        )

        budget_task = Task(
            description=f"Find budget flights, hotel options, and daily costs for {destination} within {budget}.",
            expected_output=f"✅ A full cost breakdown (flights, hotel, food, attractions) ensuring a {budget} budget is maintained.",
            tools=[search_tool],
            agent=budget_planner
        )

        itinerary_task = Task(
            description=f"Plan a 3-day itinerary for {destination} under {budget}.",
            expected_output="✅ A detailed 3-day plan, considering weather and budget constraints, with transport recommendations.",
            tools=[search_tool],
            agent=itinerary_planner
        )

        # --- Create Crew ---
        crew = Crew(
            agents=[researcher, budget_planner, itinerary_planner],
            tasks=[research_task, budget_task, itinerary_task],
            process=Process.sequential
        )

        # --- Run AI Agents ---
        responses = crew.kickoff(inputs={'destination': destination, 'budget': budget})

        # --- Display Results ---
        st.success("✅ Travel Plan Generated!")

        # Clean and display the output
        for agent_name, response in zip(["Travel Researcher", "Budget Planner", "Itinerary Planner"], responses):
            st.subheader(f"📌 {agent_name} Findings")

            # Ensure response is properly formatted
            if isinstance(response, tuple) and len(response) > 1:
                clean_response = response[1]  # Extract the actual text from tuple
            else:
                clean_response = response

            st.write(clean_response if clean_response else "No response available.")


st.markdown("🌍 *Enjoy your AI-powered trip planning!* 🚀")

Overwriting app.py


In [7]:
from pyngrok import ngrok
ngrok.kill()

outh_token = "2vImOMoSFgTUUFYxvILi7NSgqij_87QWU5QbKwPyCt8ge7rRj"
ngrok.set_auth_token(outh_token)

ngrok_tunnel = ngrok.connect(addr="5000",proto="http")
print("Tracking uri:",ngrok_tunnel.public_url)

!streamlit run --server.port 5000 app.py

Tracking uri: https://0767-34-48-148-141.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:5000
  Network URL: http://172.28.0.12:5000
  External URL: http://34.48.148.141:5000

# Agent: Travel Researcher
## Task: Find the best historical sites, weather forecast, and public transport hotels for Kerala.


# Agent: Travel Researcher
## Thought: tool_code
Thought: I need to find information on historical sites, weather, and hotels near public transport in Kerala using the available tool.  I'll break this down into separate searches.
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"top historical sites in Kerala\"}"
## Tool Output: 
{'searchParameters': {'q': 'top historical sites in Kerala', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Kerala Historic Sites & Districts to Visit (2025) - Tripadvisor', 'link': 'https://www.tripadvisor.com/Attractions-g297631-Activities-c47-t